In [1]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
import seaborn as sns
import matplotlib.pyplot as plt
from neuromaps.datasets import fetch_fslr
from surfplot import Plot

In [16]:
workdir = '/data/tu_gaor/scripts'
homedir = '/Users/ruohan/My_Repos/Cerebellum'
cwdir   = homedir
data_dir = os.path.join(cwdir, 'dat')
code_dir = os.path.join(cwdir, 'plotting')

# Dataset and yeo of interest
dataset_name = "hcpa"
yeo_of_interest = "yeo7"
term = "int" # main or int

plot_dir = os.path.join(cwdir, 'plotting/assoc_plots', yeo_of_interest)
plot_sig_only = True

In [17]:
covar_stats = pd.read_csv(os.path.join(data_dir, dataset_name, 'cc_assoc', f"{dataset_name}_covar_fusion_{yeo_of_interest}.csv"))

if plot_sig_only:
    covar_stats.loc[covar_stats["sig"].isna(), "Estimate"] = 0

# filter the cerebellar main effect rows
covar_mainc = covar_stats[~covar_stats["term"].str.contains("sr")]
covar_mainc.head()

,term,Estimate,Std. Error,t value,Pr(>|t|),DV,IV,p_fdr,sig
0,(Intercept),0.000000,0.033125,0.414120,0.678938,N1L,A1L,0.877277,NaN
1,A1L,0.000000,0.038442,0.230948,0.817436,N1L,A1L,0.949490,NaN
3,age,-0.447789,0.033985,-13.175944,0.000000,N1L,A1L,0.000000,YES
4,sexF_M,-0.093139,0.042577,-2.187547,0.029099,N1L,A1L,0.068664,yes
5,icv,0.456335,0.042309,10.785860,0.000000,N1L,A1L,0.000000,YES


In [18]:
# filter rows ending with ":sc"
covar_int = covar_stats[covar_stats["term"].str.endswith(":sr")]
covar_int.tail()

,term,Estimate,Std. Error,t value,Pr(>|t|),DV,IV,p_fdr,sig
3107,S3R:sr,0.000000,0.026228,-1.013729,0.311133,N7R,S3R,0.546311,NaN
3114,S4L:sr,0.000000,0.027483,-1.247312,0.212784,N7R,S4L,0.411654,NaN
3121,S4R:sr,0.000000,0.024966,-0.516243,0.605880,N7R,S4R,0.859881,NaN
3128,S5L:sr,0.000000,0.025619,-0.320352,0.748816,N7R,S5L,0.912589,NaN
3135,S5R:sr,-0.055433,0.026559,-2.087195,0.037304,N7R,S5R,0.086523,yes:int


In [19]:
df_map = {
    "main": covar_mainc,
    "int": covar_int
}

if term not in df_map:
    raise ValueError(f"Unknown term: {term}")

# Pivot to wide format
covar_data = df_map[term].reset_index(drop=True).pivot(index="IV", columns="DV", values="Estimate")

# Get cortical region names
cortical_names = covar_data.columns.tolist()

# Show first few rows
covar_data.head()

DV,N1L,N1R,N2L,N2R,N3L,N3R,N4L,N4R,N5L,N5R,N6L,N6R,N7L,N7R
IV,,,,,,,,,,,,,,
A1L,0.000000,0.000000,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
A1R,-0.065398,-0.090401,0.0,0.0,-0.06063,-0.07298,0.0,0.0,0.0,0.0,0.0,-0.075677,0.0,-0.070336
A2L,0.000000,-0.067180,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
A2R,0.000000,0.000000,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,-0.051750,0.0,-0.062720
A3L,0.000000,0.000000,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000


In [20]:
if plot_sig_only:
    # remove rows where all values are 0
    covar_filtered = covar_data.loc[~(covar_data == 0).all(axis=1)]
else:
    # ask user for input
    user_input = input("Enter row names to keep, separated by commas: ")
    keep_rows = [x.strip() for x in user_input.split(",")]
    covar_filtered = covar_data.loc[covar_data.index.isin(keep_rows)]

In [21]:
covar_filtered

DV,N1L,N1R,N2L,N2R,N3L,N3R,N4L,N4R,N5L,N5R,N6L,N6R,N7L,N7R
IV,,,,,,,,,,,,,,
A1R,-0.065398,-0.090401,0.0,0.0,-0.060630,-0.072980,0.0,0.0,0.0,0.0,0.0,-0.075677,0.0,-0.070336
A2L,0.000000,-0.067180,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
A2R,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,-0.051750,0.0,-0.062720
A3R,0.000000,-0.081988,0.0,0.0,-0.060690,-0.072700,0.0,0.0,0.0,0.0,0.0,-0.059210,0.0,-0.067060
D3L,0.000000,-0.077105,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,-0.058793,0.0,0.000000
M4R,0.000000,-0.059282,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-0.053789
S4L,0.000000,-0.065415,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
S5R,0.000000,0.000000,0.0,0.0,-0.060418,-0.060467,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-0.055433


In [22]:
# Yeo 7-network labels
yeo_labels_dict = {
    1: "visual",
    2: "sensory motor",
    3: "dorsal attention",
    4: "ventral attention",
    5: "limbic",
    6: "fronto parietal",
    7: "DMN"
}

In [23]:
yeo_lh = nib.load('atlases/Yeo/Yeo_JNeurophysiol11_7Networks.32k.L.label.gii')
yeo_lh_data = yeo_lh.agg_data()
yeo_rh = nib.load('atlases/Yeo/Yeo_JNeurophysiol11_7Networks.32k.R.label.gii')
yeo_rh_data = yeo_rh.agg_data()

In [24]:
# Get surfaces for visualization of all FDR weights
surfaces = fetch_fslr()
lh_surf, rh_surf = surfaces['inflated']
sulc_lh, sulc_rh = surfaces['sulc']

In [25]:
# Initialize list to store all weight arrays
yeo_weight_arrays = []

# Loop over cerebellar regions
for cereb_name in covar_filtered.index:
    row = covar_filtered.loc[cereb_name]
    
    # Create empty arrays for LH/RH
    lh_weight_yeo = np.zeros_like(yeo_lh_data, dtype=float)
    rh_weight_yeo = np.zeros_like(yeo_rh_data, dtype=float)
    
    # Assign t-values/weights to Yeo 7 networks
    for i in range(7):
        # Left hemisphere
        lh_mask = yeo_lh_data == (i + 1)
        lh_weight_yeo[lh_mask] = row[f"N{i+1}L"]
        
        # Right hemisphere
        rh_mask = yeo_rh_data == (i + 1)
        rh_weight_yeo[rh_mask] = row[f"N{i+1}R"]
    
    # Optional: check max for debugging
    lh_max = np.max(np.abs(lh_weight_yeo)) if np.any(lh_weight_yeo) else 0
    rh_max = np.max(np.abs(rh_weight_yeo)) if np.any(rh_weight_yeo) else 0
    print(f"{cereb_name}: LH max={lh_max:.3f}, RH max={rh_max:.3f}")
    
    # Store arrays
    yeo_weight_arrays.append((cereb_name, lh_weight_yeo, rh_weight_yeo))


A1R: LH max=0.065, RH max=0.090
A2L: LH max=0.000, RH max=0.067
A2R: LH max=0.000, RH max=0.063
A3R: LH max=0.061, RH max=0.082
D3L: LH max=0.000, RH max=0.077
M4R: LH max=0.000, RH max=0.059
S4L: LH max=0.000, RH max=0.065
S5R: LH max=0.060, RH max=0.060


In [26]:
# Set color scale
abs_max = np.max([
    np.max(np.abs(lh)) for _, lh, _ in yeo_weight_arrays
] + [
    np.max(np.abs(rh)) for _, _, rh in yeo_weight_arrays
])

rounded_max = np.ceil(abs_max * 10) / 10   # round UP to nearest 0.1
global_min = -rounded_max
global_max = rounded_max

print(f"global_min = {global_min}, global_max = {global_max}")

global_min = -0.1, global_max = 0.1


In [27]:
# override for specific dataset + term
if dataset_name == "hcpd" and term == "main":
    global_min = -0.3
    global_max = 0.3

# print results
print(f"global_min = {global_min}, global_max = {global_max}")

global_min = -0.1, global_max = 0.1


In [28]:
# Now create plots with the fixed min/max values
for cereb_name, lh_weight, rh_weight in yeo_weight_arrays:
    try:
        print(f"Creating plot for {cereb_name} with fixed range: {global_min} to {global_max}")

        # Initialize plot
        p = Plot(lh_surf, rh_surf)
        
        # Add sulcal layer
        p.add_layer({'left': sulc_lh, 'right': sulc_rh}, cmap='binary_r', cbar=False)
        
        # Add t-value / weight layer
        p.add_layer(
            {'left': lh_weight, 'right': rh_weight},
            cmap='coolwarm',
            color_range=(global_min, global_max),
            cbar_label='Weight'
        )

        # Save figure
        fig = p.build()
        out_fig = os.path.join(
            plot_dir, f'{dataset_name}_{yeo_of_interest}_{cereb_name}_cc_assoc_{term}.png'
        )
        fig.savefig(out_fig, dpi=300, bbox_inches='tight')
        plt.close(fig)
        print(f"  Saved surface plot: {out_fig}")

    except Exception as e:
        print(f"  Error creating surface plot: {e}")

Creating plot for A1R with fixed range: -0.1 to 0.1
  Saved surface plot: /Users/ruohan/My_Repos/Cerebellum/plotting/assoc_plots/yeo7/hcpa_yeo7_A1R_cc_assoc_int.png
Creating plot for A2L with fixed range: -0.1 to 0.1
  Saved surface plot: /Users/ruohan/My_Repos/Cerebellum/plotting/assoc_plots/yeo7/hcpa_yeo7_A2L_cc_assoc_int.png
Creating plot for A2R with fixed range: -0.1 to 0.1
  Saved surface plot: /Users/ruohan/My_Repos/Cerebellum/plotting/assoc_plots/yeo7/hcpa_yeo7_A2R_cc_assoc_int.png
Creating plot for A3R with fixed range: -0.1 to 0.1
  Saved surface plot: /Users/ruohan/My_Repos/Cerebellum/plotting/assoc_plots/yeo7/hcpa_yeo7_A3R_cc_assoc_int.png
Creating plot for D3L with fixed range: -0.1 to 0.1
  Saved surface plot: /Users/ruohan/My_Repos/Cerebellum/plotting/assoc_plots/yeo7/hcpa_yeo7_D3L_cc_assoc_int.png
Creating plot for M4R with fixed range: -0.1 to 0.1
  Saved surface plot: /Users/ruohan/My_Repos/Cerebellum/plotting/assoc_plots/yeo7/hcpa_yeo7_M4R_cc_assoc_int.png
Creating p